# Knapsack With Integer Weights問題

各々重さ$(w_\alpha \geq 0)$と価値$(c_\alpha \geq 0)$の決まったN個のアイテムがあり、そのうちのいくつかをナップサックに入れるとき、総重量$ \displaystyle W(= \sum _ {\alpha = 1} ^ {N} w_{\alpha}x_{\alpha})$をある決められた値$W_{limit}$以下に抑えながら、価値の合計$ \displaystyle C(= \sum _ {\alpha = 1} ^ {N} c_{\alpha}x_{\alpha})$を最大化するような入れ方を探す問題をナップサック問題という。ここでは重量を整数とすることにします。  

# ハミルトニアン

ハミルトニアンは次の論文を参考にします( https://arxiv.org/abs/1302.5843 )。  

$$H = H_A + H_B$$  
$$\displaystyle H_A = A \left( 1 - \sum _ { i = 1 } ^ { W_{limit} } y_i \right) ^ { 2 }+ A \left( \sum _ { i = 1 } ^ { W_{limit} } i y_i - \sum _ { \alpha } w_\alpha x_\alpha \right) ^ { 2 }$$

$$\displaystyle H_B = -B \sum _ { \alpha } c_\alpha x_\alpha$$
$H_A$は$W \leq W_{limit}$であることを保証するための制約関数、$H_B$は価値の合計を最大化するための目的関数です(価値の合計が大きいほど$H_B$は小さくなり$H$も小さくなる)。

## ハミルトニアンの詳細
ハミルトニアンには$x_\alpha$と$y_i$の二つの変数が登場します。
$x_\alpha$は$\alpha$番目のアイテムをナップサックに入れるかどうかを表す変数で、これが求めたい解そのものとなります。  
$
x_\alpha =
\begin{cases}
     1 & (\alpha \in knapsack) \\
     0 & (otherwise)
\end{cases}
$  
一方、$y_i$はアイテムの総重量を表すための変数です。$W \leq W_{limit}$という制限を課すためには総重量$W$の値が必要ですが、これは求めたい解$x_\alpha$がないと(計算結果が出ないと)知ることができません。よって$x_\alpha$によって決まる別の補助変数$y_i$を導入します。  
$
y_i =
\begin{cases}
    1 & (i = W) \\
    0 & (i \neq W)
\end{cases}
$  
$y_i$は$W_{limit}$個の要素からなり、$W$番目のみが1となるような変数です。解$\{x_\alpha\}$が決まるとそれに応じて総重量$W$が決まり、$y_i = 1$となる$i$も決まるというイメージです。$W_{limit}$の大きさに応じて$y_i$のサイズも大きくなり、必要となる補助ビットの数も増えることになります。  
$H_A$ではこの補助変数を用いて制約条件を表現しています。まず$H_A$の第一項目ですが、これは$y_i$がただ一つの$i = W$で1となるための項です。このときこの項は最小値0をとります。複数の$y_i$が1となったり、全ての$y_i$が0になったり($W \gt W_{limit}$のときは全て0ということになります)するとこの項の値は増加します。次に第二項目ですが、$\displaystyle \sum _ { i = 1 } ^ { W_{limit} } i y_i$と$\displaystyle \sum _ { \alpha } w_\alpha x_\alpha$はそれぞれ選択したアイテムの総重量を表しています。前者は少しわかりにくいですが、$i = W$の時のみ$y_i$が1なので、$0 + \dots + W + \dots + 0 = W$となります。このときこの項は最小値0をとります。$W \gt W_{limit}$のときには前者の和が$0 + \dots + 0 = 0$となり、この項は$W ^ 2$の値を取ります(値が増加します。)

# QUBO行列の構築
まずはプログラムに落とし込みやすいように$H_A$を変形します。定数項は無関係なので省略しています。

$H _ { A }$  
$ \displaystyle
=A \left\{
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right)
    +\left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right) ^ { 2 }
    +\left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) ^ { 2 }
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) \left( \sum _ \alpha w_\alpha x_\alpha \right)
    +\left( \sum _ \alpha w_\alpha x_\alpha \right) ^ 2 \right\}
$  

$ \displaystyle
= A \left\{
    \left( \sum _ { i = 1 } ^ { W_{limit} } -2 y_i \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } y_i ^ 2 \right)
   +\left( \mathop { \sum \sum } _ { i \neq j } ^ { W_{limit} } 2 y_i y_j \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } i ^ 2 y_i ^ 2 \right)
   \right.
$  

$ \displaystyle
\quad \left.
   +\left( \mathop { \sum \sum } _ {i \neq j } ^ { W_{limit} } 2ij y_i y_j \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } \sum _ { \alpha } \left( -2i w _ \alpha x _ \alpha y _ i \right) \right)
   +\left( \sum _ \alpha w_\alpha ^ 2 x_\alpha ^ 2 \right)
   +\left( \sum _ { \alpha } \sum _ { \beta } 2 w_\alpha w_\beta x_\alpha x_\beta\right )
   \right\}
$  

$ \displaystyle
= A \left\{
     \sum _ { \alpha } w_\alpha ^ 2x_\alpha ^ 2
   +\sum _ { \alpha } \sum _ { \beta } 2 w_\alpha w_\beta x_\alpha x_\beta
   +\sum _ { \alpha } \sum _ { i = 1 } ^ { W_{limit} }\left( -2 w_\alpha i \right) x_\alpha y_i
   +\sum _ { i = 1 } ^ { W_{limit} } \left( i ^ 2 - 1\right) y_i ^ 2
   +\mathop { \sum \sum } _ { i \neq j } ^ { W_{limit} } 2 \left( 1 + ij \right) y_i y_j
   \right\}
$ 

ここから実際にwildqatを使用して問題を解いていきます。
blueqatをインストールされていない方は以下のような方法でご準備ください。


In [1]:
pip install blueqat

     |████████████████████████████████| 51kB 3.7MB/s 


まず必要なライブラリをインポートします。

In [0]:
import numpy as np
import blueqat.opt as wq

アイテム用の簡単なクラス、qubo行列を計算する関数、答えを計算、表示する用の関数を用意しておきます

In [0]:
class Item():
    def __init__(self, number, weight, cost):
        self.__number = number
        self.__weight = weight
        self.__cost = cost

    @property
    def weight(self):
        return self.__weight

    @property
    def cost(self):
        return self.__cost

    def __str__(self):
        return f"#{self.__number} (weight : {self.weight}, cost : {self.cost})"

In [0]:
def get_qubo(items, wlimit, A, B):
    # qubo行列を作成
    x_size = len(items)
    y_size = wlimit
    size = x_size + y_size
    qubo = np.zeros((size, size))
    for i in range(0,size):
        for j in range(0, size):
            # 行列の下側は0のままでいい
            if i > j:
                continue
                
            wi = items[i].weight if i < x_size else 0 # 0の場合は使わないので実質的に意味はない
            wj = items[j].weight if j < x_size else 0 # 同上
            ci = items[i].cost if i < x_size else 0      # 同上
            wsum_i = i - x_size + 1
            wsum_j = j - x_size + 1

            # 対角成分
            if i == j:
                if i < x_size: # xi*xi
                    qubo[i][i] = A * wi ** 2 - B * ci
                else: # yi*yi
                    qubo[i][i] = A * (-1 + wsum_i * wsum_j)
            # 非対角成分
            else: # i < j
                if i < x_size and j < x_size: # xi*xj
                    qubo[i][j] = 2 * A * wi * wj
                elif i < x_size and j >= x_size: # xi*yj
                    qubo[i][j] = -2 * A * wi * wsum_j
                else: # yi*yj
                    qubo[i][j] = 2 * A * (1 + wsum_i * wsum_j)

    return qubo

In [0]:
def show_answer(q, items):
    print(q)
    answers = []
    weight = 0 
    cost = 0
    for i in range(len(items)):
        if q[i] > 0:
            answers.append(items[i])
            weight += items[i].weight
            cost += items[i].cost
    for answer in answers:
        print(f"selected : {answer}")
    print(f"total weight : {weight}")
    print(f"total cost : {cost}")
    print("")

In [0]:
def annealing(annealer):
    # しらみつぶしで探した最適解
    answer_opt = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
    count = 0
    found = False
    for _ in range(100):
        q = annealer.sa()
        count += 1
        print(f"count : {count}")
        show_answer(q, items)
        if q == answer_opt:
            print(f"found in {count} times")
            found = True
            break
    if(found == False):
        print("optimum solution has not been found")


アイテムのリストを用意します。軽くて安いもの(#0,#1)、軽くて高めなもの(#2,#3)、中間的なもの(#4,#5)、重くて安いもの(#6,#7)、重くて高いもの(#8,#9)をそれぞれ2つずつ用意してみました。コスパの良い#2,#3は選ばれやすく、コスパの悪い#6,#7は選ばれにくいことが予想されます。

In [0]:
items = []
items.append(Item(number=0, weight=1, cost=10))
items.append(Item(number=1, weight=2, cost=15))
items.append(Item(number=2, weight=2, cost=55))
items.append(Item(number=3, weight=3, cost=50))
items.append(Item(number=4, weight=4, cost=40))
items.append(Item(number=5, weight=5, cost=50))
items.append(Item(number=6, weight=7, cost=30))
items.append(Item(number=7, weight=8, cost=35))
items.append(Item(number=8, weight=7, cost=60))
items.append(Item(number=9, weight=8, cost=80))

$A, B$と$W_{limit}$を決めて問題を解いてみます

In [11]:
# 重さの上限(問題設定)
wlimit = 10

# コストの最大値から係数Bを決める
A = 1
cmax = max(items, key = lambda item : item.cost).cost
B = A / cmax * 0.9

# SA
annealer = wq.opt()
annealer.qubo = get_qubo(items, wlimit, A, B)
annealing(annealer)

count : 1
[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1]
selected : #3 (weight : 3, cost : 50)
selected : #7 (weight : 8, cost : 35)
selected : #9 (weight : 8, cost : 80)
total weight : 19
total cost : 165

count : 2
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
selected : #0 (weight : 1, cost : 10)
selected : #4 (weight : 4, cost : 40)
selected : #5 (weight : 5, cost : 50)
total weight : 10
total cost : 100

count : 3
[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
selected : #3 (weight : 3, cost : 50)
selected : #5 (weight : 5, cost : 50)
selected : #6 (weight : 7, cost : 30)
selected : #7 (weight : 8, cost : 35)
total weight : 23
total cost : 165

count : 4
[0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
selected : #1 (weight : 2, cost : 15)
selected : #2 (weight : 2, cost : 55)
selected : #7 (weight : 8, cost : 35)
selected : #8 (weight : 7, cost : 60)
selected : #9 (weight : 8, cost : 80)
total weight : 27
total cost : 245

co

上記のプログラムを実際に実行してみると、100回計算しても補助変数まで完全に一致した解はなかなかでないことがわかります。よくみると重量が$W_{limit}$を超えている解が得られることが多いです。詳細は略しますが、$H_A$の二つの項のバランスを調整して(両方の係数をAにするのではなく、別々に設定する)$H_A$の第一項の制約を相対的に強めることで多少改善させることも可能です。